In [ ]:
pip install langchain

In [2]:
!pip install -qU langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.8/326.8 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.9 MB/s eta 0:00:00


In [5]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

··········


In [6]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content='Hello Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 12, 'total_tokens': 22}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-007ef8c9-b518-4cec-8eaf-fa627ac86fc4-0', usage_metadata={'input_tokens': 12, 'output_tokens': 10, 'total_tokens': 22})

In [7]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm sorry, I do not know your name as I am an AI assistant and do not have access to personal information.", response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 12, 'total_tokens': 37}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-bba7573b-570b-4115-8e05-1922b99462fb-0', usage_metadata={'input_tokens': 12, 'output_tokens': 25, 'total_tokens': 37})

In [9]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can i assist you today?"),
        HumanMessage(content="What's my name?")
    ]
)

AIMessage(content='Your name is Bob.', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 35, 'total_tokens': 40}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fc4b1f0d-9618-49a6-bf36-89bb65d41f0b-0', usage_metadata={'input_tokens': 35, 'output_tokens': 5, 'total_tokens': 40})

# Message History

In [ ]:
!pip install langchain_community

In [13]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

# session_id 문자열을 받아서 BaseChatMessageHistory를 반환하는 함수
def get_session_history(session_id: str) -> BaseChatMessageHistory:
  if session_id not in store:
    store[session_id] = ChatMessageHistory()
  return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history) # 메세지 히스토리를 사용하는 모델

# refer
# https://api.python.langchain.com/en/latest/community_api_reference.html#module-langchain_community.chat_message_histories
# https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html#langchain_core.runnables.history.RunnableWithMessageHistory

In [14]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config,
)

response.content

'Hello Bob! How can I assist you today?'

In [15]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config
)

response.content

'Your name is Bob. How can I help you today, Bob?'

In [16]:
# 다른 세션 사용하기
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"I'm sorry, I do not know your name as we are communicating through text on a digital platform."

In [18]:
# 원래 세션으로 돌아오기
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [
        HumanMessage(content="What's my name?")
    ],
    config=config
)

response.content

'Your name is Bob. How can I assist you today, Bob?'

# Prompt templates

In [19]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all questions to the best of your ability."),
        MessagesPlaceholder(variable_name="messages"),  # messages 변수 값을 사용
    ]
)

chain = prompt | model

In [20]:
response = chain.invoke(
    {
        "messages": [HumanMessage(content="hi! I'm bob")] # messages 변수 값을 정의
    }
)

response.content

'Hello Bob! How can I assist you today?'

In [21]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history) # model

config = {"configurable": {"session_id": "abc5"}}

response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Jim.")],
    config=config
)

response.content

'Hello Jim! How can I assist you today?'

In [22]:
# 다시 물어보기
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config
)

response.content

'Your name is Jim.'

In [26]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all questions to the best of your ability in {language}"),  # language 변수 값을 사용
        MessagesPlaceholder(variable_name="messages"),  # messages 변수 값을 사용
    ]
)

chain = prompt | model

In [27]:
response = chain.invoke(
    {
        "messages": [HumanMessage(content="hi! I'm bob")],  # messages 변수 값을 정의
        "language": "Spanish",  # language 변수 값을 정의
    }
)

response.content

'¡Hola, Bob! ¿En qué puedo ayudarte hoy?'

In [28]:
with_message_history = RunnableWithMessageHistory(
    chain,                          # 기존 model 대신에 chain을 사용
    get_session_history,            # 메세지 히스토리
    input_messages_key="messages",  # MessagePlaceholder에서 사용하는 변수명 지정
)

In [30]:
config = {"configurable": {"session_id": "abc11"}}

response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="hi! I'm todd")],
        "language": "Spanish",
    },  # with_message_history의 첫번째 인수는 chain에 입력된다.
    config=config
)

response.content

'¡Hola Todd! ¿En qué puedo ayudarte hoy?'

In [31]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what's my name?")],
        "language": "Spanish",
    },
    config=config
)

response.content

'Tu nombre es Todd. ¿Hay algo más en lo que pueda ayudarte?'

# Managing Conversation History

- 챗봇을 만들때 이해해야할 중요한 개념은 대화 히스토리 관리 방법이다.
- 대화 히스토리를 관리하지 않으면 메세지 리스트는 제한없이 늘어나게 되고 잠재적으로 LLM의 컨텍스트 윈도우를 오버플로우할 가능성이 있다.
- 그러므로, 전달하는 메세지의 크기를 제한하는 단계를 추가하는 것이 중요하다.

In [32]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [34]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer) # prompt에 입력하기 전에 trimmer에게 전달
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)

response.content

"I'm sorry, I don't have access to personal information."

In [35]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)

response.content

'You asked what is 2 + 2.'

In [37]:
with_message_history = RunnableWithMessageHistory(
    chain,  # 메세지를 입력하고 AI로부터 출력값을 생성하는 체인을 지정 (trimmer 추가)
    get_session_history,  # 메세지 목록과, config가 입력되면, 메세지 히스토리로부터 메세지 목록을 가져온다.
    input_messages_key="messages",  # chain에 메세지 목록을 입력하기 위해 MessagePlaceholder의 메세지 키 값을 지정한다.
)

config = {"configurable": {"session_id": "abc20"}}

response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    },
    config=config
)

response.content

"I'm sorry, but I don't have access to personal information like your name. How can I assist you today?"

In [38]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config
)

response.content

"You haven't asked a math problem yet. Feel free to ask any math question you have, and I'll be happy to help you solve it."

# Streaming

- 챗봇 애플리케이션에서 중요한 UX 고려사항은 스트링(streaming)이다.
- LLM을 종종 응답값을 생성하는데 시간이 걸리기 때문에 대부분의 애플리케이션은 사용자 경험을 향상시키기 위해서 토큰이 생성될 때마다 각 토큰을 스트리밍한다.

In [40]:
config = {"configurable": {"session_id": "abc15"}}

for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm todd. tell me a joke")],
        "language": "English",
    },
    config=config
):
  print(r.content, end="|")

|Sure|,| Todd|!| Here|'s| a| joke| for| you|:

|Why| don|'t| scientists| trust| atoms|?

|Because| they| make| up| everything|!||